In [ ]:
#import sys
#!{sys.executable} -m pip install xgboost

In [64]:
import xgboost as xgb

import numpy as np
import pandas as pd
from sklearn.model_selection import KFold, train_test_split, GridSearchCV
from sklearn.metrics import confusion_matrix, mean_squared_error

In [65]:
from preprocessing_wrapper import load_preprocessed_data
from PCA import airbnb_PCA_n

pd.options.mode.chained_assignment = None  # default='warn'

data = load_preprocessed_data(pca=False)
data = data.drop(columns = ["Listing_ID", "Host_ID", "Postal_Code"])

features = data.columns.drop("Price")
target = 'Price'

data = airbnb_PCA_n(data, features, target, 80)

# Create X and Y, the target value from data
X = data.drop(columns=['Price'])
Y = data[['Price']]

In [76]:
#x_train, x_test, y_train, y_test = train_test_split(X,Y,test_size =0.2)
error = []

kf = KFold(n_splits=2, shuffle=True, random_state=rng)
for train_index, test_index in kf.split(X):
    xgb_model = xgb.XGBRegressor(gamma=5, eta=0.1, max_depth=3, n_jobs=1, eval_metric='rmse').fit(X[train_index], y[train_index])
    predictions = xgb_model.predict(X[test_index])
    rmse = mean_squared_error(y[test_index], predictions, squared = False)
    error.append(rmse)

print(error)


KeyError: "None of [Int64Index([    2,     4,     6,     8,    12,    14,    15,    20,    21,\n               22,\n            ...\n            15656, 15664, 15665, 15670, 15673, 15674, 15676, 15677, 15678,\n            15681],\n           dtype='int64', length=7841)] are in the [columns]"

In [69]:
print("Parameter optimization")
xgb_model = xgb.XGBRegressor(n_jobs=1)

params = {
        'eta': [0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9]
}

clf = GridSearchCV(xgb_model,
                   params,
                   verbose=1, n_jobs=1,)
clf.fit(X, y)
print(clf.best_score_)
print(clf.best_params_)

Parameter optimization
Fitting 5 folds for each of 10 candidates, totalling 50 fits
0.3695122726490051
{'eta': 0.1}


In [70]:
print("Parameter optimization")
xgb_model = xgb.XGBRegressor(n_jobs=1)

params = {
        'max_depth': [4,5,6],
        'gamma': [0.0001, 0.1, 1, 5],
        
}

clf = GridSearchCV(xgb_model,
                   params,
                   verbose=1, n_jobs=1,)
clf.fit(X, y)
print(clf.best_score_)
print(clf.best_params_)

Parameter optimization
Fitting 5 folds for each of 12 candidates, totalling 60 fits
0.33040114020426586
{'gamma': 5, 'max_depth': 4}
